In [1]:
from core.handler import TUserData, TEverestUser, TPodcast
from core.handler import TEverestInstallation, TCastBoxInstallation, TUserData, TEverestUser
import itertools
import pandas as pd
from core import feed_mgr
import pymongo
from collections import defaultdict
import scipy
import cPickle
import numpy as np
import util as C
reload(C)
import seaborn as sns
%matplotlib inline
from collections import Counter

# 加载用户订阅数据

In [4]:
channel_helper = C.ChannelHelper()
user_helper = C.UserHelper(channel_helper)

In [5]:
channel_helper.load_index_data()

In [6]:
idx_df = user_helper.load_subs_idx_df()

In [7]:
evst_idx_df = user_helper.load_evst_subs_idx_df()

In [8]:
idx_df.head(3)

,cn,lang,br,cs
0,jp,ja,docomo,48285
1,cn,zh,google,2781+115474
2,us,en,xiaomi,97197


In [9]:
evst_idx_df.head(3)

,uid,cs
0,d4ef183e7b34430f91707dfec7e12dd1,11414
1,6c6e4d6f97904e018b06643032d4e85d,399+2465+3308+5950+13798+49689+69215+71013+105...
2,726554550003492895e92395a8ef8943,3421+11413+11554+26353+38381+40442+52917+67079...


# 创建订阅矩阵做CF

In [11]:
%%time
mat_coo = C.build_coo_matrix(idx_df, channel_helper)

4609357it [00:27, 169434.81it/s]


CPU times: user 32.6 s, sys: 1.54 s, total: 34.1 s
Wall time: 33.7 s


In [12]:
mat_coo

<4609357x137344 sparse matrix of type '<type 'numpy.float32'>'
	with 42015431 stored elements in COOrdinate format>

In [13]:
%%time
mat_csr = mat_coo.tocsr()

CPU times: user 28.4 s, sys: 1.08 s, total: 29.5 s
Wall time: 29.4 s


In [14]:
evst_mat_coo = C.build_coo_matrix(evst_idx_df, channel_helper)

1884585it [00:10, 180934.03it/s]


In [15]:
%%time
evst_mat_csr = evst_mat_coo.tocsr()

CPU times: user 8.34 s, sys: 396 ms, total: 8.74 s
Wall time: 8.71 s


In [16]:
%%time
C.save_mat(C.USER_ITEM_MAT_PATH, mat_csr, compressed=False)
C.save_mat(C.EVST_USER_ITEM_PATH, evst_mat_csr, compressed=False)

CPU times: user 644 ms, sys: 1.05 s, total: 1.7 s
Wall time: 4.89 s


# 创建用户(locale)订阅矩阵

In [17]:
idx_df['locale'] = idx_df['lang'].astype(str)  + '/' + idx_df['cn'].astype(str)

In [18]:
idx_df.head(5)

,cn,lang,br,cs,locale
0,jp,ja,docomo,48285,ja/jp
1,cn,zh,google,2781+115474,zh/cn
2,us,en,xiaomi,97197,en/us
3,cn,en,google,9781+123162,en/cn
4,us,en,verizon,5307+43037,en/us


In [30]:
locale_dist = idx_df['locale'].value_counts()
locales = locale_dist[locale_dist >= 200].index

In [31]:
locales[:10]

Index([u'en/us', u'en/gb', u'pt/br', u'en/ca', u'en/au', u'sv/se', u'ru/ru',
       u'ko/kr', u'de/de', u'fr/fr'],
      dtype='object')

In [32]:
len(locales)

311

In [33]:
user_helper.save_locale_data(locales)

In [34]:
user_helper.load_locale_data()

In [35]:
idx_df['locale_code'] = idx_df['locale'].apply(lambda x: user_helper.get_locale_code(x))

In [36]:
idx_df.head(3)

,cn,lang,br,cs,locale,locale_code
0,jp,ja,docomo,48285,ja/jp,10
1,cn,zh,google,2781+115474,zh/cn,25
2,us,en,xiaomi,97197,en/us,0


In [37]:
user_locale_mat_coo = scipy.sparse.coo_matrix((np.ones(mat_csr.shape[0], dtype = np.float32),
                                      (np.arange(mat_csr.shape[0]),
                                       idx_df['locale_code'].values)), shape = (mat_csr.shape[0], len(locales) + 1))

In [38]:
locale_item_mat_coo = np.dot(user_locale_mat_coo.T, mat_coo)

In [39]:
locale_item_mat_csr = locale_item_mat_coo.tocsr()

In [40]:
C.save_mat(C.LOCALE_ITEM_MAT_PATH, locale_item_mat_csr, compressed=False)